In [43]:
import pandas as pd
import numpy as np
from pathlib import Path
from scipy.stats import chi2_contingency

In [ ]:
## Obtener el dataframe crudo 

def find_project_root(start: Path) -> Path:
    for p in [start, *start.parents]:
        if (p / "data" / "raw").exists() and (p / "notebooks").exists():
            return p
    return start

ROOT = find_project_root(Path.cwd())
RAW = ROOT / "data" / "raw"
PROCESSED = ROOT / "data" / "processed"
REPORTS = ROOT / "reports" / "tables"

df_raw = pd.read_csv(RAW / "survey_results_public.csv", low_memory=False)

print(df_raw.shape)
display(df_raw.head(3))


(65437, 114)


,ResponseId,MainBranch,Age,Employment,RemoteWork,Check,CodingActivities,EdLevel,LearnCode,LearnCodeOnline,...,JobSatPoints_6,JobSatPoints_7,JobSatPoints_8,JobSatPoints_9,JobSatPoints_10,JobSatPoints_11,SurveyLength,SurveyEase,ConvertedCompYearly,JobSat
0,1,I am a developer by profession,Under 18 years old,"Employed, full-time",Remote,Apples,Hobby,Primary/elementary school,Books / Physical media,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,I am a developer by profession,35-44 years old,"Employed, full-time",Remote,Apples,Hobby;Contribute to open-source projects;Other...,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",Books / Physical media;Colleague;On the job tr...,Technical documentation;Blogs;Books;Written Tu...,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
2,3,I am a developer by profession,45-54 years old,"Employed, full-time",Remote,Apples,Hobby;Contribute to open-source projects;Other...,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",Books / Physical media;Colleague;On the job tr...,Technical documentation;Blogs;Books;Written Tu...,...,NaN,NaN,NaN,NaN,NaN,NaN,Appropriate in length,Easy,NaN,NaN


In [45]:
audit = pd.DataFrame({
    "dtype": df_raw.dtypes.astype(str),
    "missing_%": (df_raw.isna().mean()*100).round(2),
    "unique": df_raw.nunique(dropna=True)
}).sort_values("missing_%", ascending=False)
audit = audit.reset_index().rename(columns={"index": "column"})
display(audit.head(30))
print("Duplicados:", df_raw.duplicated().sum())


,column,dtype,missing_%,unique
0,AINextMuch less integrated,object,98.25,286
1,AINextLess integrated,object,96.40,249
2,AINextNo change,object,80.90,539
3,AINextMuch more integrated,object,79.46,700
4,EmbeddedAdmired,object,74.43,1053
5,EmbeddedWantToWorkWith,object,73.10,1471
6,EmbeddedHaveWorkedWith,object,66.05,1630
7,ConvertedCompYearly,float64,64.19,6113
8,AIToolNot interested in Using,object,62.69,1531
9,AINextMore integrated,object,62.67,884


Duplicados: 0


In [46]:
## Obtener esquema de los datos para poder tener información de las columnas

schema = pd.read_csv(RAW / "survey_results_schema.csv")
schema = schema.rename(columns={
    "qname": "column",
    "type": "question_type"
})

schema.head()


,qid,column,question,force_resp,question_type,selector
0,QID2,MainBranch,Which of the following options best describes ...,True,MC,SAVR
1,QID127,Age,What is your age?*,True,MC,SAVR
2,QID296,Employment,Which of the following best describes your cur...,True,MC,MAVR
3,QID308,RemoteWork,Which best describes your current work situation?,False,MC,SAVR
4,QID341,Check,Just checking to make sure you are paying atte...,True,MC,SAVR


In [47]:
## Mezclar para obtener mejor información de las columnas

profile = audit.merge(
    schema,
    on="column",
    how="left"
)

profile.shape
profile.sample(100, random_state=42)


,column,dtype,missing_%,unique,qid,question,force_resp,question_type,selector
80,OfficeStackAsyncHaveWorkedWith,object,26.50,6014,NaN,NaN,NaN,NaN,NaN
4,EmbeddedAdmired,object,74.43,1053,NaN,NaN,NaN,NaN,NaN
40,WorkExp,float64,54.68,51,QID288,How many years of working experience do you have?,False,Slider,HSLIDER
69,AIThreat,object,31.71,3,QID338,Do you believe AI is a threat to your current ...,False,MC,SAVR
10,Knowledge_9,object,57.77,5,QID289,I have asked my employer to reimburse me for t...,NaN,MC,MAVR
...,...,...,...,...,...,...,...,...,...
52,AIComplex,object,43.42,5,QID343,How well do the AI tools you use in your devel...,False,MC,SAVR
21,Frequency_2,object,56.65,5,QID290,Interacting with people outside of your immedi...,NaN,MC,MAVR
2,AINextNo change,object,80.90,539,NaN,NaN,NaN,NaN,NaN
23,ProfessionalCloud,object,56.46,3,QID332,"At my company, our applications, databases and...",False,MC,SAVR


In [48]:
## Descargar en un csv para elegir las columnas finales con comodidad

profile_sorted = profile.sort_values(["missing_%", "unique"], ascending=[True, True])
REPORTS.mkdir(parents=True, exist_ok=True)
profile_sorted.to_csv(REPORTS / "profile_sorted_by_missing.csv", index=False)


In [49]:
#Columnas finales (a mantener)
FINAL_COLS = [
    "ResponseId",
    "MainBranch",
    "Employment",
    "Country",
    "RemoteWork",
    "ConvertedCompYearly",
    "YearsCodePro",
    "YearsCode",
    "DevType",
    "EdLevel",
    "Age",
    "OrgSize",
    "LanguageHaveWorkedWith",
    "DatabaseHaveWorkedWith",
    "WebframeHaveWorkedWith",
    "PlatformHaveWorkedWith",
    "ToolsTechHaveWorkedWith",
]

present = [c for c in FINAL_COLS if c in df_raw.columns]

df_final = df_raw[present].copy()
df_final.shape

(65437, 17)

In [50]:
## Nuevas variables

if "ConvertedCompYearly" in df_final.columns:
    df_final["has_salary"] = df_final["ConvertedCompYearly"].notna()
    df_final["log_salary"] = np.log1p(df_final["ConvertedCompYearly"])

if "YearsCodePro" in df_final.columns:
    map_years = {"Less than 1 year": 0.5, "More than 50 years": 50}
    df_final["YearsCodePro_clean"] = df_final["YearsCodePro"].replace(map_years)
    df_final["YearsCodePro_num"] = pd.to_numeric(df_final["YearsCodePro_clean"], errors="coerce")



## Snapshot de el dataset final 
print("Shape final:", df_final.shape)
print("\nMissing % (top 10):")
(df_final.isna().mean().sort_values(ascending=False) * 100).round(2).head(10)

Shape final: (65437, 21)

Missing % (top 10):


ConvertedCompYearly        64.19
log_salary                 64.19
PlatformHaveWorkedWith     35.26
WebframeHaveWorkedWith     30.99
OrgSize                    27.44
DatabaseHaveWorkedWith     23.20
YearsCodePro_clean         21.13
YearsCodePro               21.13
YearsCodePro_num           21.13
ToolsTechHaveWorkedWith    19.80
dtype: float64

In [51]:
# Guardar dataset limpio
PROCESSED.mkdir(parents=True, exist_ok=True)

output_path = PROCESSED / "df_final_v1.csv"
df_final.to_csv(output_path, index=False)
print(f"Saved: {output_path.resolve()}")


Saved: /media/camilo-perez/New Volume3/Proyectos/devstats/data/processed/df_final_v1.csv


In [52]:
## RemoteWork vs has_salary

remote_counts = pd.crosstab(
    df_final["RemoteWork"].fillna("Missing"),
    df_final["has_salary"],
    margins=True
)
display(remote_counts)

has_salary,False,True,All
RemoteWork,,,
"Hybrid (some remote, some in-person)",13116,9899,23015
In-person,7023,3937,10960
Missing,10623,8,10631
Remote,11240,9591,20831
All,42002,23435,65437


In [53]:
## Country (Top 15) vs has_salary (proporciones por columna)

top_countries = df_final["Country"].value_counts().head(15).index

country_counts = pd.crosstab(
    df_final.loc[df_final["Country"].isin(top_countries), "Country"],
    df_final["has_salary"],
    margins=True
)
display(country_counts)

has_salary,False,True,All
Country,,,
Australia,750,510,1260
Brazil,690,685,1375
Canada,1233,871,2104
France,1192,918,2110
Germany,2901,2046,4947
India,3194,1037,4231
Italy,801,540,1341
Netherlands,888,561,1449
Poland,948,586,1534


In [54]:
def chi2_and_cramers_v(contingency: pd.DataFrame):
    """
    Calcula Chi-cuadrado de independencia y tamaño de efecto Cramér's V.
    Retorna: chi2, p, dof, n, cramers_v
    """
    chi2, p, dof, expected = chi2_contingency(contingency)
    n = contingency.to_numpy().sum()
    r, k = contingency.shape
    cramers_v = np.sqrt((chi2 / n) / (min(r - 1, k - 1)))
    return chi2, p, dof, n, cramers_v


In [55]:
tab_remote = pd.crosstab(
    df_final["RemoteWork"],
    df_final["has_salary"]
)

chi2_r, p_r, dof_r, n_r, v_r = chi2_and_cramers_v(tab_remote)

print("RemoteWork vs has_salary")
print(f"chi2 = {chi2_r:.2f}, p = {p_r:.3e}, dof = {dof_r}, n = {n_r}, Cramer's V = {v_r:.3f}")


RemoteWork vs has_salary
chi2 = 301.69, p = 3.077e-66, dof = 2, n = 54806, Cramer's V = 0.074


In [56]:
tab_country = pd.crosstab(
    df_final.loc[df_final["Country"].isin(top_countries), "Country"],
    df_final["has_salary"]
)

chi2_c, p_c, dof_c, n_c, v_c = chi2_and_cramers_v(tab_country)

print("Country(top15) vs has_salary")
print(f"chi2 = {chi2_c:.2f}, p = {p_c:.3e}, dof = {dof_c}, n = {n_c}, Cramer's V = {v_c:.3f}")


Country(top15) vs has_salary
chi2 = 876.18, p = 5.477e-178, dof = 14, n = 40406, Cramer's V = 0.147


In [57]:
results_bias = pd.DataFrame({
    "Contraste": ["RemoteWork vs has_salary", "Country(top15) vs has_salary"],
    "chi2": [chi2_r, chi2_c],
    "p": [p_r, p_c],
    "Cramers_V": [v_r, v_c],
    "n": [n_r, n_c],
    "dof": [dof_r, dof_c]
})


results_bias_display = results_bias.copy()
results_bias_display["chi2"] = results_bias_display["chi2"].map(lambda x: f"{x:.2f}")
results_bias_display["p"] = results_bias_display["p"].map(lambda x: f"{x:.3e}")
results_bias_display["Cramers_V"] = results_bias_display["Cramers_V"].map(lambda x: f"{x:.3f}")

display(results_bias_display)


,Contraste,chi2,p,Cramers_V,n,dof
0,RemoteWork vs has_salary,301.69,3.077e-66,0.074,54806,2
1,Country(top15) vs has_salary,876.18,5.477e-178,0.147,40406,14
